In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet, Ridge
from scipy.stats import zscore, mstats
from constrained_linear_regression import ConstrainedLinearRegression

In [2]:
def rolling_regression_sklearn_advanced(data, rolling_window, n_step_ahead=1, 
                                        l1_ratio=0.1, 
                                        dropna=False, remove_outliers=False, 
                                        winsorize=False, winsorize_limits=(0.05, 0.95),
                                        fit_intercept=False, min_coef=None, max_coef=None):
    """
    Perform rolling regression from sklearn with additional data processing options.
    
    Parameters:
        data (pd.DataFrame): DataFrame where the last column is the target variable. 
                             Should have a DateTimeIndex.
        rolling_window (int): Number of samples to use for each regression.
        n_step_ahead (int, optional): Number of steps ahead to predict. Default is 1.
        l1_ratio (float, optional): The L1 regularization ratio. Default is 0.1.
        dropna (bool, optional): Whether to drop NaN values. Default is False.
        remove_outliers (bool, optional): Whether to remove outliers based on Z-score. Default is False.
        winsorize (bool, optional): Whether to winsorize data. Default is False.
        winsorize_limits (tuple, optional): Percentiles for winsorizing. Default is (0.05, 0.95).
    
    Returns:
        pd.DataFrame: Coefficients for each window.
        pd.Series: Predictions.
    """
    # Drop NaN values if requested
    if dropna:
        data = data.dropna()
    
    n_samples, n_features_plus_one = data.shape
    n_features = n_features_plus_one - 1
    
    coefs = pd.DataFrame(index=data.index, columns=[f'coef_{i}' for i in range(n_features)])
    predictions = pd.Series(index=data.index, name='predictions')
    
    for start in range(0, n_samples - rolling_window - n_step_ahead + 1, n_step_ahead):
        window = data.iloc[start:start + rolling_window].copy()  # Use copy to avoid SettingWithCopyWarning
        
        # Remove outliers if requested
        if remove_outliers:
            z_scores = np.abs(zscore(window))
            window = window[(z_scores < 3).all(axis=1)]
        
        # Winsorize data if requested
        if winsorize:
            window = window.apply(lambda col: mstats.winsorize(col, limits=winsorize_limits), axis=0)
        
        # X, y = window.iloc[:, :-1], window.iloc[:, -1]
        X, y = window.drop('target', axis=1), window['target']
        # model = make_pipeline(StandardScaler(),
        #                       Ridge(alpha = l1_ratio, fit_intercept=False),

        #                      )
        # scale_ = model.named_steps['standardscaler'].scale_
        # coefs.iloc[end_idx] = model.named_steps['ridge'].coef_ / scale_


        model = ConstrainedLinearRegression(ridge=l1_ratio, normalize=True, fit_intercept=fit_intercept)
        model.fit(X, y, min_coef=min_coef, max_coef=max_coef)

        end_idx = start + rolling_window

        coefs.iloc[end_idx] = model.coef_
        future_X = data.iloc[end_idx:end_idx + n_step_ahead, :-1]
        future_preds = model.predict(future_X)
        predictions.iloc[end_idx:end_idx + n_step_ahead] = future_preds
        
    return pd.concat([coefs, predictions], axis=1)


# Assuming we have a DataFrame 'data' with the appropriate format, this function could be called as follows:
# results = rolling_regression_sklearn_advanced(data, rolling_window=20)
# But here, we will not run it as we don't have a predefined 'data' DataFrame.


np.random.seed(20)
data_df = pd.DataFrame()
SIZE = 120
data_df['data_1'] = np.random.normal(size=((SIZE)))
data_df['data_2'] = np.random.normal(size=((SIZE)))
data_df['target'] = data_df['data_1'] * 1.5 + data_df['data_2'] * 0.5 + np.random.normal(scale=0.2, size=((SIZE)))
data_df['date'] = pd.date_range(start='1/1/2018', periods=len(data_df), freq='H')
data_df = data_df.set_index('date')

N_MACRO = 3
MACRO_COLUMNS = ['Fed', 'GDP', 'VIX']
df_regime = pd.DataFrame(np.random.normal(loc=0.1,size=(SIZE, N_MACRO)), index=data_df.index)
df_regime.columns = MACRO_COLUMNS[:N_MACRO]
df_regime
def helper_regime(df_regime, q_list):
    df_regimec = df_regime.copy()
    for i, col in enumerate(df_regime):
        q = q_list[i]
        df_regimec[col] = np.ceil(df_regimec[col].rank(pct=True)*q)
    return df_regimec

label_dict = {'Fed': {1.0 : 'Hiking', 2.0: 'Cutting'},
              'GDP': {1.0: 'Accelerating', 2.0: 'Slowing', 3.0: 'Falling'},
              'VIX': {1.0: 'High', 2.0: 'Medium', 3.0: 'Low'},
              }

def get_df_regime_label(df_regime_discrete, label_dict):
    df_regime_labelled = df_regime_discrete.copy()
    for col, label_sub_dict in label_dict.items():
        value_counts = df_regime_discrete[col].value_counts()
        for value, label in label_sub_dict.items():
            new_col = df_regime_labelled[col].replace(value, f'{col}_{label}_{value_counts[value]}')
            df_regime_labelled[col] = new_col
    return df_regime_labelled

df_regime_discrete = helper_regime(df_regime, [2, 3, 3])        
df_regime_labelled = get_df_regime_label(df_regime_discrete, label_dict)
df_regime_labelled


,Fed,GDP,VIX
date,,,
2018-01-01 00:00:00,Fed_Hiking_60,GDP_Slowing_40,VIX_Low_40
2018-01-01 01:00:00,Fed_Hiking_60,GDP_Falling_40,VIX_Low_40
2018-01-01 02:00:00,Fed_Hiking_60,GDP_Accelerating_40,VIX_High_40
2018-01-01 03:00:00,Fed_Cutting_60,GDP_Slowing_40,VIX_High_40
2018-01-01 04:00:00,Fed_Cutting_60,GDP_Falling_40,VIX_Low_40
...,...,...,...
2018-01-05 19:00:00,Fed_Hiking_60,GDP_Accelerating_40,VIX_High_40
2018-01-05 20:00:00,Fed_Cutting_60,GDP_Accelerating_40,VIX_Medium_40
2018-01-05 21:00:00,Fed_Cutting_60,GDP_Slowing_40,VIX_High_40


In [4]:
pd.concat([df_regime_labelled, data_df], axis=1).groupby('Fed').mean()['data_1']
test = [pd.concat([df_regime_labelled, data_df], axis=1).groupby(col).mean()['data_1'] for col in MACRO_COLUMNS[:N_MACRO]]
pd.DataFrame(pd.concat(test)).reset_index()

C:\Users\Wazir\AppData\Local\Temp\ipykernel_10896\3924581582.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pd.concat([df_regime_labelled, data_df], axis=1).groupby('Fed').mean()['data_1']
C:\Users\Wazir\AppData\Local\Temp\ipykernel_10896\3924581582.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test = [pd.concat([df_regime_labelled, data_df], axis=1).groupby(col).mean()['data_1'] for col in MACRO_COLUMNS[:N_MACRO]]
C:\Users\Wazir\AppData\Local\Temp\ipykernel_10896\3924581582.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric

,index,data_1
0,Fed_Cutting_60,-0.003344
1,Fed_Hiking_60,-0.094704
2,GDP_Accelerating_40,0.014295
3,GDP_Falling_40,0.072788
4,GDP_Slowing_40,-0.234155
5,VIX_High_40,-0.142273
6,VIX_Low_40,0.070370
7,VIX_Medium_40,-0.075168


In [5]:
# Testing the function and measuring execution time
rolling_window = 100
n_step_ahead = 1
start_time_advanced = time.time()
df_results = rolling_regression_sklearn_advanced(
    data_df, rolling_window=rolling_window, n_step_ahead=n_step_ahead,
    dropna=False, remove_outliers=False, winsorize=False, winsorize_limits=(0.01, 0.99)
)
end_time_advanced = time.time()
time_advanced = end_time_advanced - start_time_advanced
time_advanced


C:\Users\Wazir\AppData\Local\Temp\ipykernel_10896\1021464922.py:32: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  predictions = pd.Series(index=data.index, name='predictions')
c:\Users\Wazir\anaconda3\envs\regtool\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but ConstrainedLinearRegression was fitted without feature names
  warnings.warn(
c:\Users\Wazir\anaconda3\envs\regtool\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but ConstrainedLinearRegression was fitted without feature names
  warnings.warn(
c:\Users\Wazir\anaconda3\envs\regtool\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but ConstrainedLinearRegression was fitted without feature names
  warnings.warn(
c:\Users\Wazir\anaconda3\envs\regtool\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but ConstrainedLi

0.199859619140625

In [6]:
df_shap = pd.concat([df_results, data_df], axis=1)
df_shap['shap_1'] = df_shap['coef_0'] * df_shap['data_1']
df_shap['shap_2'] = df_shap['coef_1'] * df_shap['data_2']
df_shap['pred_shap'] = df_shap.filter(regex='shap').sum(axis=1)
df_shap

,coef_0,coef_1,predictions,data_1,data_2,target,shap_1,shap_2,pred_shap
date,,,,,,,,,
2018-01-01 00:00:00,NaN,NaN,NaN,0.883893,0.289559,1.315804,NaN,NaN,0.000000
2018-01-01 01:00:00,NaN,NaN,NaN,0.195865,-0.470209,-0.155492,NaN,NaN,0.000000
2018-01-01 02:00:00,NaN,NaN,NaN,0.357537,1.605993,1.213914,NaN,NaN,0.000000
2018-01-01 03:00:00,NaN,NaN,NaN,-2.343262,-0.153662,-3.807196,NaN,NaN,0.000000
2018-01-01 04:00:00,NaN,NaN,NaN,-1.084833,-1.786169,-2.264937,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...
2018-01-05 19:00:00,1.505092,0.485731,-1.576530,-1.334921,0.890717,-1.768914,-2.009179,0.432649,-1.576530
2018-01-05 20:00:00,1.507695,0.484474,1.134139,0.676640,0.235252,1.398751,1.020166,0.113973,1.134139
2018-01-05 21:00:00,1.509077,0.486443,1.209178,0.831106,-0.092560,0.996478,1.254203,-0.045025,1.209178


In [7]:
shap = np.array([1.0, -0.5])
pred = shap.sum()
pred


0.5

In [8]:
np.abs(shap)

array([1. , 0.5])

In [10]:
def feature_engineering(df, transformations):
    '''
    Extends the dataframe with engineered features based on the specified transformations.
    
    Parameters:
        df (pd.DataFrame): The original dataframe.
        transformations (dict): A dictionary with keys as column names and values as lists of transformations.
    
    Returns:
        pd.DataFrame: The dataframe with added engineered features.
    '''
    engineered_df = df.copy()
    
    for column, transformation_list in transformations.items():
        monthly = True if column[-2:] == '_m' else False
        for transformation in transformation_list:
            operation, *params = transformation  # Unpack operation and parameters
            
            if operation in ['mean', 'std', 'kurt', 'pct_change', 'diff']:
                window = params[0]
                lag = params[1] if len(params) > 1 else 0  # Use lag if provided, else default to 0
                if monthly:
                    monthly_series = df[column][df[column].diff() != 0].copy()
                    if operation in ['mean', 'std', 'kurt']:
                        rolled_series = getattr(monthly_series.rolling(window=window), operation)()
                    elif operation == 'pct_change':
                        rolled_series = monthly_series.pct_change(periods=window).reindex(df.index)
                    elif operation == 'diff':
                        rolled_series = monthly_series.diff(periods=window).reindex(df.index)
                    engineered_df[f"{column}_{operation}_{window}d_lag{lag}"] = rolled_series.shift(lag).reindex(df.index).ffill()
                else:
                    if operation in ['mean', 'std', 'kurt']:
                        rolled_series = getattr(df[column].rolling(window=window), operation)()
                    elif operation == 'pct_change':
                        rolled_series = df[column].pct_change(periods=window)
                    elif operation == 'diff':
                        rolled_series = df[column].diff(periods=window)
                    engineered_df[f"{column}_{operation}_{window}d_lag{lag}"] = rolled_series.shift(lag)

            elif operation == 'lag':
                lag = params[0]
                if monthly:
                    monthly_series = df[column][df[column].diff() != 0].copy()
                    engineered_df[f"{column}_lag{lag}"] = monthly_series.shift(lag).reindex(df.index).ffill()
                else:
                    engineered_df[f"{column}_lag{lag}"] = df[column].shift(lag)

    return engineered_df


# def fea

example_df = pd.DataFrame({"X1": range(10),
                           'X2_m': [1, 1, 1, 2, 2, 2, 3, 3, 4, 4]})
transformations_new = { 'X1': [['lag', 1]],
                        'X2_m': [['lag', 1], ['lag', 2], ['diff', 1, 1], ['pct_change', 1, 1]]
                        }

df = feature_engineering(example_df, transformations_new)
df

,X1,X2_m,X1_lag1,X2_m_lag1,X2_m_lag2,X2_m_diff_1d_lag1,X2_m_pct_change_1d_lag1
0,0,1,NaN,NaN,NaN,NaN,NaN
1,1,1,0.0,NaN,NaN,NaN,NaN
2,2,1,1.0,NaN,NaN,NaN,NaN
3,3,2,2.0,1.0,NaN,NaN,NaN
4,4,2,3.0,1.0,NaN,1.0,1.000000
5,5,2,4.0,1.0,NaN,1.0,1.000000
6,6,3,5.0,2.0,1.0,1.0,1.000000
7,7,3,6.0,2.0,1.0,1.0,0.500000
8,8,4,7.0,3.0,2.0,1.0,0.500000
9,9,4,8.0,3.0,2.0,1.0,0.333333


In [11]:
def feature_engineering_updated(df, transformations):
    '''
    Extends the dataframe with engineered features based on the specified transformations.
    
    Parameters:
        df (pd.DataFrame): The original dataframe.
        transformations (dict): A dictionary with keys as column names and values as lists of transformations.
    
    Returns:
        pd.DataFrame: The dataframe with added engineered features.
    '''
    engineered_df = df.copy()
    
    for column, transformation_list in transformations.items():
        for transformation in transformation_list:
            operation, window = transformation  # Unpack operation and parameters
            if operation in ['mean', 'std', 'kurt', 'pct_change', 'diff']:
                if operation in ['mean', 'std', 'kurt']:
                    transformed_series = getattr(df[column].rolling(window=window), operation)()
                elif operation == 'pct_change':
                    transformed_series = df[column].pct_change(periods=window)
                elif operation == 'diff':
                    transformed_series = df[column].diff(periods=window)
                engineered_df[f"{column}_{operation}_{window}d"] = transformed_series
    return engineered_df


example_df = pd.DataFrame({"X1": range(10),
                           'X2_m': [1, 1, 1, 2, 2, 2, 3, 3, 4, 4]})
transformations_new = { 'X1': [['mean', 1]],
                        'X2_m': [['diff', 1], ['pct_change', 1]]
                        }

feature_engineering_updated(example_df, transformations_new)


def lag_variables(df, lag_dict):
    dfc = df.copy()
    for col, windows in lag_dict.items():
        for window in windows:
            dfc[f'{col}_lag{window}'] = dfc[col].shift(window)

    return dfc

,X1,X2_m,X1_mean_1d_,X2_m_diff_1d_,X2_m_pct_change_1d_
0,0,1,0.0,NaN,NaN
1,1,1,1.0,0.0,0.000000
2,2,1,2.0,0.0,0.000000
3,3,2,3.0,1.0,1.000000
4,4,2,4.0,0.0,0.000000
5,5,2,5.0,0.0,0.000000
6,6,3,6.0,1.0,0.500000
7,7,3,7.0,0.0,0.000000
8,8,4,8.0,1.0,0.333333
9,9,4,9.0,0.0,0.000000


In [20]:
df_monthly = pd.read_excel('../Data/sample_data.xlsx', skiprows=1)
df_monthly.columns = ['Date', 'CPI-U_SA', 'CPI-U_NSA', 'Manheim_Used_Vehicle', 'PPI_Used_Vehicles', 'PPI_Automotive', 'PPI_Passenger_Cars', '5y_Treasury']
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'])
df_monthly.set_index('Date', inplace=True)


: 

In [19]:
df_monthly

,CPI-U_SA,CPI-U_SA,Manheim_Used_Vehicle,PPI_Used_Vehicles,PPI_Automotive,PPI_Passenger_Cars,5y_Treasury
Date,,,,,,,
2000-01-31,153.900,153.900,109.1,NaN,NaN,133.400,6.58
2000-02-29,153.000,152.998,109.5,NaN,NaN,132.700,6.68
2000-03-31,153.000,152.975,110.1,NaN,NaN,132.700,6.50
2000-04-30,154.000,153.975,109.8,NaN,NaN,132.800,6.26
2000-05-31,155.400,155.385,110.2,NaN,NaN,133.500,6.69
...,...,...,...,...,...,...,...
2023-06-30,198.746,202.007,215.1,113.417,241.519,139.972,3.95
2023-07-31,196.086,201.624,211.7,105.340,241.128,140.277,4.14
2023-08-31,193.671,198.768,212.2,107.378,242.503,140.669,4.31
